# Loading Data

## QUOKKA Dataset

QUOKKA is a simulation dataset based on the AMReX/BoxLib framework—similar to Orion, Nyx, Maestro, Castro, IAMR, and WarpX. This yt frontend supports QUOKKA by mapping native fields to derived fields with proper physical units.

To load a QUOKKA dataset, simply run:

```python
import yt
ds = yt.load("plt007")
```

Native gas fields (e.g., `('boxlib', 'gasDensity')`) are converted to derived fields (e.g., `('gas', 'density')`). Particle data is auto-detected from `*_particles` directories, and metadata is parsed from the `Header` and `metadata.yaml` files.

**Note:** If `metadata.yaml` is missing, ensure it is provided for proper parameter extraction.

### Example Usage

```python
import yt

ds = yt.load("path_to_quokka_dataset")
print(ds.parameters.get('HydroMethod', 'Unknown'))  # Expected: 'Quokka'
yt.SlicePlot(ds, 'z', ('gas', 'density')).show()
```
